# KNN

K近邻算法(KNN)是一种基本分类与回归的方法,也是经典机器学习中最简单的一种.K近邻法的输入为实例的特征向量,对应于特征空间的点;输出为实例的类别,可以取多类.K近邻法假设给定一个训练数据集,其中的实例类别已定.分类时，对新的实例,根据K各最近邻的训练实例的类别,通过多数表决等方式进行预测.因此,K近邻不具有显示的学习过程.



### 例子:

使用KNN对电影题材进行分类

电影可以按照题材进行分类,然而题材本身是如何定义的?由谁来判断某部电影属于哪个题材?也就是说同一题材电影有哪些公共的**特征**,比如动作片会有很多的打斗镜头,爱情片会有很多的接吻镜头,恐怖片会有很多的惊悚镜头.但是,不同题材的电影特征也会有交叉,比如动作片也会有爱情片中的镜头(比如接吻),爱情片也会有很多的动作镜头(比如接吻),那么我们应该如何使用现有的特征对电影进行题材分类？

下面这个例子中含有6部电影的镜头数据集:

![](picture/36.png)

在表中 **of kicks,of kisses**为特征(futures),**Type of movie**为标签(labels).

**注意:**

- 一般如果需要使用标签进行学习的话,那么我们可以称之为**监督学习**,如果算法学习不需要标签,我们称之为**非监督学习**.由于KNN比较特殊,我们可以将其归类为监督学习

在这个例子中,我们如何使用现有的数据集将新进来的未知电影名进行题材分类?

![](picture/37.png)

我们如果要使用KNN算法将上述案例中的电影进行分类,那么我们需要先看看KNN算法的核心是如何进行的,实际上KNN算法的核心很简单即为**距离度量**

### 1-距离度量:

特征空间中两个实例点的距离是两个实例**相似度的反映**.KNN模型的特征空间一般是n维实数向量空间$R^n$.使用的距离是欧式距离,但是也可以是其他距离,如更一般的$L_p$距离($L_p$ distance),或Minkowski距离.

特征空间X是n维实数向量空间$R^n$,$x_i,x_j \in X,x_i=(x_i^{(1)},x_i^{(2)},x_i^{(3)},...x_i^{(n)})^T,x_j=(x_j^{(1)},x_j^{(2)},x_j^{(3)},...x_j^{(n)})^T,x_i,x_j$的$L_p$距离定义为 

### $L_p(x_i,x_j)=(\sum_{l=1}^{n}|x_i^{(l)}-x_j^{(l)}|^p)^P$

当 $p\geqslant 1$. P=2时,称为欧式距离(Euclidean distance),即

### $L_2(x_i,x_j)=(\sum_{l=1}^{n}|x_i^{(l)}-x_j^{(l)}|^2)^{\frac{1}{2}}$

当P=1,为曼哈顿距离(Manhattan distance),即

### $L_1(x_i,x_j)=(\sum_{l=1}^{n}|x_i^{(l)}-x_j^{(l)}|)$

当$P=\infty $,它是各个坐标距离的最大值,即

### $L_{\infty}(x_i,x_j)=\underset{l} max |x_i^{(l)}-x_j^{(l)}|$

**注意:**

- 不同的距离度量所确定的邻近点是不同的

### 2-K的选择:

K值的选择会对结果产生重大的影响,所以在KNN中,K就是一个需要人为调节的参数
- K选择的太小,会在较小的领域中进行选择,这样容易发生过拟合,比如k=1时,为最近邻算法.
- K选择的太大,同样会产生欠拟合
- K值最好选择的是奇数个,容易在K个结果中去"投票"
- K值的选择一般比较小

### 3- KNN Algorithm

下面给出KNN算法

**KNN:**

输入:训练样本集$T=\{(x_1,y_1),(x_2,y_2),...,(x_N,y_N)\}$


其中,$x_i \in X \subseteq R^n$为实例的特征向量,$y_i \in Y=\{c_1,c_2,...,c_k\}$为实例的类别,$i=1,2...N$;实例特征向量x;


输出:


实例x所属的类y

(1) 根据给定的距离度量,在训练集T中找出与x最近邻的k个点,涵盖着K个点的x的领域记为$N_k(x)$;

(2) 在$N_k(x)$中根据决策规则(如多数表决)决定x的类别y;

### $y=\underset{c_j} argmax \sum_{x_i \in N_k(x)} I(y_i=c_i),i=1,2,...N; j=1,2,..K$

其中$I$为指示函数,即当$y_i=c_j$时$I$为1，否则为0.


### 4-电影分类代码

In [1]:
import numpy as np

#### 4.1 加载数据集

In [2]:
def loadData():
    """
    loading data.
    
    Note:
    -----
       1. 在机器学习中,我们经常使用的是向量的形式.
       2. 由于KNN中labels不参与计算,所以我们可以将labels定义为字符形式,一般在其他情况下,我们会将Y定义为1,0或者1,-1的形式在二分类中.
       
    Return:
    -------
        trainset and labels
    
    """
    X = [[3,104],
        [2,100],
        [1,81],
        [101,10],
        [99,5],
        [98,2]]
    
    Y = ['Romance','Romance','Romance','Action','Action','Action']
    
    return np.array(X),np.array(Y)
    
    
    
    

In [3]:
X,y = loadData()

#### 4.2 建立模型

In [4]:
def KNN(X,y,data,K):
    """
    将传入未知电影的特征与现有的训练特征进行KNN计算.
    
    Parameters:
    ----------
        X:trainset
        y:labels
        data: test set
        K:parameter K in KNN model.
        
    Return:
    ------
        classify result.
        
    """
    
    # calculate euclidean distance
    Euclidean_distance = np.power(np.sum((data - X)**2,axis=1),0.5)
    
    # get "short distance" labels.
    sort_k = np.argsort(Euclidean_distance)[:K]  # real number of k.
    get_K_y = y[sort_k]
    
    
    # calculate label Probability in get_K_y.
    prob_dict = {}
    
    for label in get_K_y:
        if label not in prob_dict:
            prob_dict[label] = 1
        else:
            prob_dict[label] += 1
    
    predict_y = sorted(prob_dict.items(),key=lambda z:z[1],reverse=True)[0]
    
    print('predict result is {} and the probability is {}.'.format(predict_y[0],predict_y[1] / len(get_K_y)))
    
    return predict_y[0]
    
    

In [5]:
data = np.array([[18,90]])
KNN(X,y,data,3)

predict result is Romance and the probability is 1.0.


'Romance'

我们可以看到,在使用KNN的时候,并不存在学习参数,而是每次有新进来的数据都需要计算距离,这对于小样本而言是无伤大碍的,但是对于大数据集而言就显得计算量过大,比如10张高清图片.

### 5-约会网站测试KNN

现在给予一份约会网站的数据,按照已有的数据集对VIP进行人选的推荐.

#### 5.1 加载数据集

In [6]:
def loadData(path):
    """
    load data
    
    Return:
    ------
        1.Train_data is numpy.
        2.labels; as same as numpy.
    """
    Train_data = []
    labels = []
    np.random.shuffle
    
    with open(path) as f:
        
        original_data = f.readlines()
        
    for data in original_data:
        data_split = data.strip().split('\t')
        train_ = data_split[:-1]
        label_ = data_split[-1]
        Train_data.append(train_)
        labels.append(label_)
    
    
    return np.array(Train_data,dtype='float'),np.array(labels)

In [7]:
path = "data_set/datingTestSet.txt"

X,y = loadData(path=path)

X与y的形状

In [8]:
X.shape

(1000, 3)

In [9]:
y.shape

(1000,)

#### 5.2 特征处理

在开始建立model之前,我们需要观测一下原数据,在原数据中第一个特征下的值明显高于后面两个特征,这样的数值大小差异会使得对结果的影响或者说权重影响很大,所以我们需要做一些数据预处理操作,也可以叫做[特征工程](https://www.zhihu.com/question/29316149),我们这里使用的方式是归一化,有些地方的英文也叫标准化(normal).

归一化:

对于每个特征下

### $newValue  = \frac{(oldValue - minValue)}{maxValue - minValue}$

In [10]:
def normal(X):
    """
    Normalization data.
    
    Return:
    ------
        newVlue: Normal'data.
    """
    minValue = np.min(X,axis=0)
    maxValue = np.max(X,axis=0)

    newValue = (X - minValue) / (maxValue - minValue)
    
    return newValue

In [11]:
Norm_X = normal(X)
Norm_X

array([[0.44832535, 0.39805139, 0.56233353],
       [0.15873259, 0.34195467, 0.98724416],
       [0.28542943, 0.06892523, 0.47449629],
       ...,
       [0.29115949, 0.50910294, 0.51079493],
       [0.52711097, 0.43665451, 0.4290048 ],
       [0.47940793, 0.3768091 , 0.78571804]])

#### 5.3 数据集划分

现在数据已经归一化了,这样可以避免权重的影响.那么现在将数据集划分为训练样本和测试样本,实际上我们应该划分的样本应该是训练样本,验证样本与测试样本,由于这里的数据量较小,所以我们只划分训练样本和测试样本,划分数据集的作用是**检测模型的可靠性.**

In [12]:
def split_data(X,y,pre):
    
    # permutation: shuffling x and return index if you input real number.
    shuffle_index = np.random.permutation(X.shape[0])
    shuffle_X = X[shuffle_index,:]
    shuffle_y = y[shuffle_index]
    
    split_index = np.int(pre * X.shape[0])
    train_x = shuffle_X[:split_index]
    train_y = shuffle_y[:split_index]
    
    test_x = shuffle_X[split_index:]
    test_y = shuffle_y[split_index:]
    return train_x,train_y,test_x,test_y
    

In [13]:
train_x,train_y,test_x,test_y = split_data(Norm_X,y,0.8)

In [14]:
train_x.shape

(800, 3)

In [15]:
test_x.shape

(200, 3)

当然你也可以使用[scikit-learn](https://scikit-learn.org/)进行数据划分

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
X_train, X_test, y_train, y_test = train_test_split(Norm_X,y,test_size = 0.2)

#### 5.4 Model

现在一切准备工作已经完毕,我们开始着手建立模型,并使用测试集去检测正确率

In [18]:
def KNN_model(X_train, X_test, y_train, y_test,K):
    """
    Implementation KNN model.
    
    parameters:
    ----------
        1. X_train: train set.
        2. X_test: test set.
        3. y_train: train set labels.
        4. y_test: test set labels.
        5. K: real number, parameter of KNN model.
        
    Return:
    ------
        
        Test set accuracy.
    """
    
    accuracy = 0
    m,n = X_test.shape  # shape at test set .
    
    for i in range(m):
        
        data = X_test[i]
        # calculate euclidean distance
        Euclidean_distance = np.power(np.sum((data - X_train)**2,axis=1),0.5)

        # get "short distance" labels.
        sort_k = np.argsort(Euclidean_distance)[:K]  # real number of k.
        
        get_K_y = y_train[sort_k]
        
        # calculate label Probability in get_K_y.
        prob_dict = {}

        
        for label in get_K_y:
            if label not in prob_dict:
                prob_dict[label] = 1
            else:
                prob_dict[label] += 1

        predict_y = sorted(prob_dict.items(),key=lambda z:z[1],reverse=True)[0][0]
        
        # checking predict y is right?
        if predict_y == y_test[i]:
            accuracy += 1
    
    print('The test set accurate is {}'.format(accuracy / len(y_test)))
    
    
    
    
    

In [19]:
KNN_model(X_train, X_test, y_train, y_test,5)

The test set accurate is 0.945


### 6 - Summary

在KNN中需要调控的参数只有K,以及距离的P项(但是在此案例中没有,感兴趣的可以使用不同的距离进行尝试),这是KNN的优势相比于其他模型.然而KNN的缺陷也很明显,那就是每次进入一个新的样本就要与所有的训练样本进行计算,这样也是的计算力度与所耗费的时间太大.所以在日常中我们很少会使用到KNN,但是其思想是非常牛逼的,在后面学到的算法,我们会发现很多都会用到"距离"的思想.

# Homework

实现手写数字识别系统.

数据集在data_set文件夹下的handwriting.zip

Good Luck~~